****Librerías****

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
sns.set()
import json
import ast
from pandas.io.json import json_normalize

In [3]:
movies = pd.read_csv(r'Dataset\movies_dataset.csv', sep=',')
movies.head(2)

C:\Users\Windows\AppData\Local\Temp\ipykernel_8116\557888431.py:1: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  movies = pd.read_csv(r'Dataset\movies_dataset.csv', sep=',')


,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0


****Transformaciones****

***Eliminar las columnas que no serán utilizadas, video,imdb_id,adult,original_title,poster_path y homepage.****

In [4]:
columnas = ['video','imdb_id','adult','original_title','poster_path','homepage']
movies.drop(columns= columnas, inplace=True)

***Los valores nulos del campo release date deben eliminarse.***

In [5]:
#Verifico cuantos nulos y na existen
print(movies['release_date'].isnull().sum())
print(movies['release_date'].isna().sum())

87
87


In [6]:
movies = movies.dropna(subset=['release_date'])

In [7]:
#Verifico nuevamente
print(movies['release_date'].isnull().sum())
print(movies['release_date'].isna().sum())

0
0


***Los valores nulos de los campos revenue, budget deben ser rellenados por el número 0.***

In [8]:
#La columna 'budget' tiene valores tipo str.  Se cambian a 0.
movies['budget'] = movies['budget'].replace('N/A', value= 0)

In [9]:
#Luego completo los N/A por 0.
movies['revenue'] = movies['revenue'].fillna(value=0)
movies['budget'] = movies['budget'].fillna(value=0)

***De haber fechas, deberán tener el formato AAAA-mm-dd, además deberán crear la columna release_year donde extraerán el año de la fecha de estreno.***

In [10]:
#Verifico qué datos hay en la columna. Parece haber algunos datos que no son reconocidos como fechas.
movies['release_date'].describe()

count          45379
unique         17336
top       2008-01-01
freq             136
Name: release_date, dtype: object

In [11]:
# Aplico el método pd.to_datetime con el parámetro errors='coerce' para que, si hay algún dato que no esté en formato fecha, lo reconozca como NaT.
movies['release_date'] = pd.to_datetime(movies['release_date'], errors='coerce')

In [12]:
#Verifico qué tipo de datos quedaron. Efectivamente, ahora sí todos los datos son reconocidos como fechas.
movies['release_date'].describe()

C:\Users\Windows\AppData\Local\Temp\ipykernel_8116\571393109.py:2: FutureWarning: Treating datetime data as categorical rather than numeric in `.describe` is deprecated and will be removed in a future version of pandas. Specify `datetime_is_numeric=True` to silence this warning and adopt the future behavior now.
  movies['release_date'].describe()


count                   45376
unique                  17333
top       2008-01-01 00:00:00
freq                      136
first     1874-12-09 00:00:00
last      2020-12-16 00:00:00
Name: release_date, dtype: object

In [13]:
#Ahora borro los registros NaT
movies = movies.dropna(subset= ['release_date'])

In [14]:
movies['release_date'].isna().sum()

0

In [15]:
#Por último, creo lla columna release_year
movies['release_year'] = movies['release_date'].dt.year

In [16]:
movies['release_year']

0        1995
1        1995
2        1995
3        1995
4        1995
         ... 
45460    1991
45462    2011
45463    2003
45464    1917
45465    2017
Name: release_year, Length: 45376, dtype: int64

***Crear la columna con el retorno de inversión, llamada return con los campos revenue y budget, dividiendo estas dos últimas revenue / budget, cuando no hay datos disponibles para calcularlo, deberá tomar el valor 0.***

In [17]:
#La columna budget contiene valores que no son int
movies['budget'].describe()

count     45376
unique     1223
top           0
freq      36490
Name: budget, dtype: object

In [18]:
#Convierto los valores a float
movies['budget'] = movies['budget'].astype(float)

In [19]:
#Creo la columna
movies['return'] = movies['revenue'] / movies['budget']

In [20]:
#Reemplazo los valores NaN e infinito (cuando budget = 0) por 0
movies['return'] = movies['return'].fillna(0)
movies['return'] = movies['return'].replace(to_replace= np.inf, value=0)

In [21]:
movies['return']

0        12.451801
1         4.043035
2         0.000000
3         5.090760
4         0.000000
           ...    
45460     0.000000
45462     0.000000
45463     0.000000
45464     0.000000
45465     0.000000
Name: return, Length: 45376, dtype: float64

***Algunos campos, como belongs_to_collection, production_companies y otros (ver diccionario de datos) están anidados, esto es o bien tienen un diccionario o una lista como valores en cada fila, ¡deberán desanidarlos para poder y unirlos al dataset de nuevo hacer alguna de las consultas de la API! O bien buscar la manera de acceder a esos datos sin desanidarlos.***

Las columnas que tienen diccionarios son:
* belongs_to_collection => Diccionarios
* genres => Lista de diccionarios
* prooduction_companies => Lista de diccionarios
* production_countries => Lista de diccionarios (con algunos registros vacíos o con númeroos)
* spoken_languajes => Lista de diccionarios

Columna 'belongs_to_collection':

Los diccionarios anidados tienen la siguiente información:

"{'id': 10194, 'name': 'Toy Story Collection', 'poster_path': '/7G9915LfUQ2lVfwMEEhDsn3kT4B.jpg', 'backdrop_path': '/9FBwqcd9IRruEDUrTdcaafOMKUq.jpg'}"

De la cual, la única info útil es 'name'.

Entonces, al desanidar el diccionario, voy a crear una columna que sólo contenga laa información que hay en 'name'.

In [22]:
# Definimos una función que reciba como parámetro:
#1) El nombre de la columna
#2) El nombre de la clave que nos interesa guardar



def desanidado(columna):
    

    import re #Regular expressión

    #Definimos una expresión regular (re) patrón para extraer de la colección de la columna 'belongs_to_collection'. 
    #Esta re patrón permite verificar si una cadena de caracteres coincide con ella. 
    #En este caso, queremos que:
    #1) La cadena de caracteres a extraer comience con 'name': (que es la clave que nos interesa)
    #2) Dentro de [] están los caracteres que quiero matchear
    #3) ^ significa que va a matchear cualquier caracter excepto '
    #4) * significa que el patrón se puede repetir 0, 1 o más veces.
    pattern = r"'name': '([^']*)'"

    #Extraemos el nombre de la colección y la almaceno en una nueva columna 'collection_name'
    movies[columna] = movies[columna].str.extract(pattern, flags=re.IGNORECASE) #IGNORECASE ignora la diferencia entre mayúscula y minúscula

    #el resultado se almacena en la columna collection_name
    #luego, elimino 'belongs_to_collection'

    return movies.head(2)

In [23]:
desanidado('belongs_to_collection')

,belongs_to_collection,budget,genres,id,original_language,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,release_year,return
0,Toy Story Collection,30000000.0,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",862,en,"Led by Woody, Andy's toys live happily in his ...",21.946943,"[{'name': 'Pixar Animation Studios', 'id': 3}]","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,7.7,5415.0,1995,12.451801
1,NaN,65000000.0,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",8844,en,When siblings Judy and Peter discover an encha...,17.015539,"[{'name': 'TriStar Pictures', 'id': 559}, {'na...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,6.9,2413.0,1995,4.043035


Columna 'genres':

Tiene una lista de diccionarios, y necesito sacar todos los 'names' que figuran en cada diccionario. Por lo tanto, el proceso es distinto al anterior.

In [24]:
#Verifico si tiene valor nan o null

print(movies['genres'].isna().sum())
print(movies['genres'].isnull().sum())

0
0


In [25]:
# convertimos la cadena JSON en una lista de diccionarios
movies["genres"] = movies["genres"].apply(ast.literal_eval)

In [26]:
#Aplico una función lambda para extraer los valores de 'name' para cada diccionario x en genre.
movies['genres'] = movies['genres'].apply(lambda x: [genre['name'] for genre in x])

In [27]:
movies['genres'].head()

0     [Animation, Comedy, Family]
1    [Adventure, Fantasy, Family]
2               [Romance, Comedy]
3        [Comedy, Drama, Romance]
4                        [Comedy]
Name: genres, dtype: object

Columna 'production_companies':

Tiene una lista de diccionarios, igual que genres.

In [28]:
#Verifico si tiene valor nan o null

print(movies['production_companies'].isna().sum())
print(movies['production_companies'].isnull().sum())

0
0


In [29]:
# convertimos la cadena JSON en una lista de diccionarios
movies['production_companies'] = movies['production_companies'].apply(ast.literal_eval)

In [30]:
#Aplico una función lambda para extraer los valores de 'name' para cada diccionario x en company.
movies['production_companies'] = movies['production_companies'].apply(lambda x: [company['name'] for company in x])


In [31]:
movies['production_companies'].head()


0                            [Pixar Animation Studios]
1    [TriStar Pictures, Teitler Film, Interscope Co...
2                       [Warner Bros., Lancaster Gate]
3             [Twentieth Century Fox Film Corporation]
4         [Sandollar Productions, Touchstone Pictures]
Name: production_companies, dtype: object

Columna 'production_countries':

Tiene una lista de diccionarios, igual que genres.

In [32]:
#Verifico si tiene valor nan o null

print(movies['production_countries'].isna().sum())
print(movies['production_countries'].isnull().sum())

0
0


In [33]:
# convertimos la cadena JSON en una lista de diccionarios
movies['production_countries'] = movies['production_countries'].apply(ast.literal_eval)

In [34]:
#Aplico una función lambda para extraer los valores de 'name' para cada diccionario x en company.
movies['production_countries'] = movies['production_countries'].apply(lambda x: [country['iso_3166_1'] for country in x])

In [35]:
movies['production_countries'].head()

0    [US]
1    [US]
2    [US]
3    [US]
4    [US]
Name: production_countries, dtype: object

Columna 'spoken_leanguajes':

Tiene una lista de diccionarios, igual que genres.

In [36]:
#Verifico si tiene valor nan o null

print(movies['spoken_languages'].isna().sum())
print(movies['spoken_languages'].isnull().sum())

0
0


In [37]:
# convertimos la cadena JSON en una lista de diccionarios
movies['spoken_languages'] = movies['spoken_languages'].apply(ast.literal_eval)

In [38]:
#Aplico una función lambda para extraer los valores de 'name' para cada diccionario x en company.
movies['spoken_languages'] = movies['spoken_languages'].apply(lambda x: [country['iso_639_1'] for country in x])

In [39]:
movies['spoken_languages'].head()

0        [en]
1    [en, fr]
2        [en]
3        [en]
4        [en]
Name: spoken_languages, dtype: object

****Verifico los duplicado y los elimino si hay****

In [40]:
movies['id'].describe()

count      45376
unique     45346
top       141971
freq           3
Name: id, dtype: object

In [41]:
movies['id'] = movies['id'].drop_duplicates()

In [42]:
movies['id'].describe()

count     45346
unique    45346
top         862
freq          1
Name: id, dtype: object

In [43]:
movies.head()

,belongs_to_collection,budget,genres,id,original_language,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,release_year,return
0,Toy Story Collection,30000000.0,"[Animation, Comedy, Family]",862,en,"Led by Woody, Andy's toys live happily in his ...",21.946943,[Pixar Animation Studios],[US],1995-10-30,373554033.0,81.0,[en],Released,NaN,Toy Story,7.7,5415.0,1995,12.451801
1,NaN,65000000.0,"[Adventure, Fantasy, Family]",8844,en,When siblings Judy and Peter discover an encha...,17.015539,"[TriStar Pictures, Teitler Film, Interscope Co...",[US],1995-12-15,262797249.0,104.0,"[en, fr]",Released,Roll the dice and unleash the excitement!,Jumanji,6.9,2413.0,1995,4.043035
2,Grumpy Old Men Collection,0.0,"[Romance, Comedy]",15602,en,A family wedding reignites the ancient feud be...,11.7129,"[Warner Bros., Lancaster Gate]",[US],1995-12-22,0.0,101.0,[en],Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,6.5,92.0,1995,0.000000
3,NaN,16000000.0,"[Comedy, Drama, Romance]",31357,en,"Cheated on, mistreated and stepped on, the wom...",3.859495,[Twentieth Century Fox Film Corporation],[US],1995-12-22,81452156.0,127.0,[en],Released,Friends are the people who let you be yourself...,Waiting to Exhale,6.1,34.0,1995,5.090760
4,Father of the Bride Collection,0.0,[Comedy],11862,en,Just when George Banks has recovered from his ...,8.387519,"[Sandollar Productions, Touchstone Pictures]",[US],1995-02-10,76578911.0,106.0,[en],Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,5.7,173.0,1995,0.000000


In [44]:
movies.to_csv('Movies ETL.csv')